# Práctica 4

**Nombre:** Beatriz Domínguez Urías

**e-mail:** beatriz.dominguez0182@alumnos.udg.mx

## MODULES

In [1]:
import math
import numpy as np
import pandas as pd
import panel as pn

import panel.widgets as pnw
import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

pn.extension("plotly")
pn.config.throttled = True
#pn.config.theme = "dark"

## CLASSES

In [2]:
class PlotUtility:
  """Class to simplify creation of plots with plotly graph objects module
  """
  def __init__(self, plot_title: str):
    """Create a graph object figure to be used for plots in this utility

    :param plot_title: Main title of the plot
    :type plot_title: str
    """
    self.plot_title = plot_title
    self.figure = go.Figure()

  def scatter_plot(self, plot_name: str, x, y, z=None, marker_size:int=2, line_size:int=2):
    """Plot a scatter graph object of either 2 or 3 dimensions.

    :param plot_name: Title of the graph object line
    :type plot_name: str
    :param x: Elements of x
    :type x: _type_
    :param y: Elements of y
    :type y: _type_
    :param z: Elements of z if not provided then a 2D plot is generated, defaults to None
    :type z: _type_, optional
    :param marker_size: Size of marker, defaults to 2
    :type marker_size: int, optional
    :param line_size: Size of line, defaults to 2
    :type line_size: int, optional
    """
    if z is not None:
      self.figure.add_trace(go.Scatter3d(x = list(x),
                                         y = list(y),
                                         z = list(z),
                                         marker = dict(size=marker_size),
                                         line = dict(width=line_size),
                                         mode = "lines",
                                         name = plot_name,
                                         showlegend = True))
    else:
      self.figure.add_trace(go.Scatter(x = list(x),
                                       y = list(y),
                                       marker = dict(size=marker_size),
                                       line = dict(width=line_size),
                                       mode = "lines",
                                       name = plot_name,
                                       showlegend = True))

  def get_figure(self):
    """Get graph object figure

    :return: Graph object figure
    :rtype: _type_
    """
    return self.figure
  
  def show_plot(self):
    """Show plot of graph object
    """
    self.figure.show()
     
  def update_axis(self, xlabel:str=None, ylabel:str=None, zlabel:str=None):
    """Update graph object labels and ranges

    :param xlabel: Label of x axis, defaults to None
    :type xlabel: str, optional
    :param ylabel: Label of y axis, defaults to None
    :type ylabel: str, optional
    :param zlabel: Label of z axis, defaults to None
    :type zlabel: str, optional
    """
    self.figure.update_layout(title = self.plot_title,
                              scene = dict(xaxis = dict(title= xlabel),
                                           yaxis = dict(title= ylabel),
                                           zaxis = dict(title= zlabel)))
    
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [3]:
def bm_2d(n_steps:int=1000, speed:int=6, start_pos:list=[0,0])-> pd.DataFrame:
  """Generate a brownian motion trajectory

  :param n_steps: Number of steps, defaults to 1000
  :type n_steps: int, optional
  :param speed: Speed, defaults to 6
  :type speed: int, optional
  :param start_pos: Start position, defaults to [0,0]
  :type start_pos: list, optional
  :return: Dataframe with x and y positions of trajectory
  :rtype: pd.DataFrame
  """  
  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':start_pos[0], 'y_pos':start_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

def correlated_random_walker(cauchy_exponent:float, n_steps:int=1000, speed:int=6, start_pos:list=[0,0])-> pd.DataFrame:
  """Generate a correlated random walk trajectory

  :param cauchy_exponent: Cauchy distribution exponent of range (0,1)
  :type cauchy_exponent: float
  :param n_steps: Number of steps, defaults to 1000
  :type n_steps: int, optional
  :param speed: Speed, defaults to 6
  :type speed: int, optional
  :param start_pos: Start position, defaults to [0,0]
  :type start_pos: list, optional
  :return: Dataframe with x and y positions of trajectory
  :rtype: pd.DataFrame
  """  
  # Init velocity vector
  velocity = Vec2d(speed, 0)

  # Init Correlated Random Walk dataframe
  CRW_2d_df = pd.DataFrame(columns = ["x_pos", "y_pos"])
  temp_df = pd.DataFrame([{"x_pos":start_pos[0], "y_pos": start_pos[1]}])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    # Define turn angle following a cauchy distribution
    turn_angle = wrapcauchy.rvs(cauchy_exponent)
    velocity = velocity.rotated(turn_angle)
    # Populate dataframe with walker displacement
    temp_df = pd.DataFrame([{"x_pos":CRW_2d_df.x_pos[i]+velocity.x, "y_pos": CRW_2d_df.y_pos[i]+velocity.y}])
    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  return CRW_2d_df

def generate_levy_flight(alpha:float, n_steps:int=10000, speed:int=1, miu:int=0, beta:float=1.0, cauchy_exponent:float=0.7, start_pos:list=[0,0])-> pd.DataFrame:
  """Generate a levy flight trajectory

  :param alpha: Levy stable distribution alpha exponent of range (0,2]
  :type alpha: float
  :param n_steps: Number of steps, defaults to 10000
  :type n_steps: int, optional
  :param speed: Speed, defaults to 1
  :type speed: int, optional
  :param miu: Offset of distribution, defaults to 0
  :type miu: int, optional
  :param beta: Levy stable distribution beta exponent, defaults to 1.0
  :type beta: float, optional
  :param cauchy_exponent: Cauchy distribution exponent of range (0,1), defaults to 0.7
  :type cauchy_exponent: float, optional
  :param start_pos: Start position, defaults to [0,0]
  :type start_pos: list, optional
  :return: Dataframe with x and y positions of trajectory
  :rtype: pd.DataFrame
  """  
  # Init velocity vector
  velocity = Vec2d(speed, 0)

  # Init Lévy Flight dataframe
  levy_df = pd.DataFrame(columns = ["x_pos", "y_pos"])
  temp_df = pd.DataFrame([{"x_pos":start_pos[0], "y_pos": start_pos[1]}])
  levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

  # Init steps control variables
  levy_steps = 1
  step_count = 0

  for i in range(n_steps-1):
    if step_count >= levy_steps:
      # Define turn angle following a cauchy distribution
      turn_angle = wrapcauchy.rvs(cauchy_exponent)
      velocity = velocity.rotated(turn_angle)

      # Populate dataframe with walker displacement
      temp_df = pd.DataFrame([{"x_pos":levy_df.x_pos[i]+velocity.x, "y_pos": levy_df.y_pos[i]+velocity.y}])
      levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

      # Reset steps control variables
      levy_steps = levy_stable.rvs(alpha=alpha, beta=beta, loc=miu)
      step_count = 0
    else:
      # Increase a step without a change on direction
      step_count += 1
      temp_df = pd.DataFrame([{"x_pos":levy_df.x_pos[i]+velocity.x, "y_pos": levy_df.y_pos[i]+velocity.y}])
      levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

  return levy_df

def get_euclidean_distance(point_one: np.ndarray, point_two: np.ndarray) -> float:
    """Get euclidean distance between two points

    :param point_one: x and y component of point one
    :type point_one: np.ndarray
    :param point_two: x and y component of point two
    :type point_two: np.ndarray
    :return: Distance
    :rtype: float
    """       
    x_comp = math.pow(point_two[0] - point_one[0],2)
    y_comp = math.pow(point_two[1] - point_one[1],2)
    distance = math.sqrt(x_comp + y_comp)
  
    return distance

def get_path_length(trajectory: pd.DataFrame) -> pd.DataFrame:
  """Get path length of a trajectory

  :param trajectory: Trajectory used to calculate path length
  :type trajectory: pd.DataFrame
  :return: Dataframe with path length
  :rtype: pd.DataFrame
  """  
  trajectory = trajectory.to_numpy()
  aux_path_length = np.empty(shape=(0))

  for i in range(1, trajectory.shape[0]):
    distance = get_euclidean_distance(trajectory[i-1], trajectory[i])
    aux_path_length = np.append(aux_path_length, distance)
  
  PL_df = pd.DataFrame(np.cumsum(aux_path_length), columns= ["distance"])
  return PL_df

def get_mean_squared_displacement(trajectory: pd.DataFrame) -> pd.DataFrame:
  """Get mean squared displacement of a trajectory

  :param trajectory: Trajectory used to calculate mean squared displacement
  :type trajectory: pd.DataFrame
  :return: Dataframe with mean squared displacement
  :rtype: pd.DataFrame
  """  
  N = trajectory.shape[0]
  trajectory = trajectory.to_numpy()
  aux_MSD = np.empty(shape=(0))

  for tau in range(1,N):
      displacement_vec = np.array([math.pow(get_euclidean_distance(trajectory[i-tau], trajectory[i]), 2)
                          for i in range(tau, trajectory.shape[0],1)])
      aux_MSD = np.append(aux_MSD, np.mean(displacement_vec))
      displacement_vec = np.empty(shape=(0))

  MSD_df = pd.DataFrame(aux_MSD, columns= ["MSD"])
  return MSD_df

## DASHBOARD

### Widgets for parameters

In [4]:
widg_traj = pnw.RadioButtonGroup(name="Trajectory type", options=["BM", "CRW", "LF"], width=10)
widg_steps = pnw.EditableIntSlider(name="Number of steps", value=1000, step=10, start=1, end=10000, width=235)
widg_x_pos = pnw.IntInput(name="Start pos of X", value=0, step=1, start=-100, end=100, width=100)
widg_y_pos = pnw.IntInput(name="Start pos of Y", value=0, step=1, start=-100, end=100, width=100)
widg_speed = pnw.EditableIntSlider(name="Speed", value=6, step=1, start=1, end=100, width=235)
widg_cauchy = pnw.FloatInput(name="Cauchy exponent", value=0.7, step=0.1, start=0.1, end=0.99, width=100)
widg_levy_alpha = pnw.FloatInput(name="Levy exponent", value=1.2, step=0.1, start=0.1, end=2, width=100)
widg_levy_beta = pnw.FloatInput(name="Beta", value=0, step=0.1, start=-1.0, end=1.0, width=100)
widg_metric = pnw.Select(name="Metric type", options=["PL", "MSD"], width=100)

### Interactive Functions

In [5]:
def update_layout(traj_type:str) -> pn.layout.Column:
    """Function to update column layout based in selected trajectory

    :param value: Trajectory type (BM, CRW, LF)
    :type value: str
    :return: Column layout with parameters needed for trajectory type
    :rtype: pn.layout.Column
    """    
    pos_row = pn.Row(widg_x_pos, widg_y_pos)
    base_column = pn.Column("### Parameters", pn.Row(widg_steps), pos_row, pn.Row(widg_speed))#, styles=dict(background='WhiteSmoke'))
    
    if traj_type == "BM":
        updated_column = base_column
    elif traj_type == "CRW":
        updated_column = pn.Column(base_column, pn.Row(widg_cauchy))#, styles=dict(background='WhiteSmoke'))
    elif traj_type == "LF":
        updated_column = pn.Column(base_column, pn.Row(widg_cauchy), pn.Row(widg_levy_alpha, widg_levy_beta))#, styles=dict(background='WhiteSmoke'))
    
    return updated_column

def plot_trajectory(traj_type:str, n_steps:int, x_pos:int, y_pos:int, speed:int, cauchy_exp:float, levy_exp:float, levy_beta:float) -> go.Figure:
    """Generate trajectory with selected parameters and plot graph object

    :param traj_type: Trajectory type (BM, CRW, LF)
    :type traj_type: str
    :param n_steps: Number of steps
    :type n_steps: int
    :param x_pos: Start position of x
    :type x_pos: int
    :param y_pos: Start position of y
    :type y_pos: int
    :param speed: Speed
    :type speed: int
    :param cauchy_exp: Cauchy distribution exponent of range (0,1)
    :type cauchy_exp: float
    :param levy_exp: Levy stable distribution alpha exponent of range (0,2]
    :type levy_exp: float
    :param levy_beta: Levy stable distribution beta exponent
    :type levy_beta: float
    :return: Figure where plot is updated
    :rtype: go.Figure
    """    
    global g_trajectory
    temp_fig = PlotUtility(f"{traj_type} trajectory")
    
    if traj_type == "BM":
        g_trajectory = bm_2d(n_steps=n_steps, speed=speed, start_pos=[x_pos, y_pos])
    elif traj_type == "CRW":
        g_trajectory = correlated_random_walker(cauchy_exponent=cauchy_exp, n_steps=n_steps, speed=speed, start_pos=[x_pos, y_pos])
    elif traj_type == "LF":
        g_trajectory = generate_levy_flight(alpha=levy_exp, n_steps=n_steps, speed=speed, beta=levy_beta, cauchy_exponent=cauchy_exp, start_pos=[x_pos, y_pos])

    temp_fig.scatter_plot(plot_name=f"{traj_type}", x=g_trajectory.x_pos, y=g_trajectory.y_pos, z=g_trajectory.index)
    temp_fig.update_axis("x_pos", "y_pos", "time")
    fig_traj = temp_fig.get_figure()
    
    return fig_traj

def plot_metric(metric_type:str, traj_type:str, n_steps:int, x_pos:int, y_pos:int, speed:int, cauchy_exp:float, levy_exp:float, levy_beta:float) -> go.Figure:
    """Plot graph object figure of selected metric for current trajectory, where only metric type is considered 
    for generation of the metric, while the rest are used to monitor changes of current trajectory

    :param metric_type: Metric type (MSD, PL)
    :type metric_type: str
    :param traj_type: Trajectory type (BM, CRW, LF)
    :type traj_type: str
    :param n_steps: Number of steps
    :type n_steps: int
    :param x_pos: Start position of x
    :type x_pos: int
    :param y_pos: Start position of y
    :type y_pos: int
    :param speed: Speed
    :type speed: int
    :param cauchy_exp: Cauchy distribution exponent of range (0,1)
    :type cauchy_exp: float
    :param levy_exp: Levy stable distribution alpha exponent of range (0,2]
    :type levy_exp: float
    :param levy_beta: Levy stable distribution beta exponent
    :type levy_beta: float
    :return: Figure where plot is updated
    :rtype: go.Figure
    """    
    temp_fig = PlotUtility(f"{metric_type} metric")

    if metric_type == "MSD":
        current_metric = get_mean_squared_displacement(g_trajectory)
        temp_fig.scatter_plot(plot_name=f"{metric_type}", x=current_metric.index, y=current_metric.MSD)
    else:
        current_metric = get_path_length(g_trajectory)
        temp_fig.scatter_plot(plot_name=f"{metric_type}", x=(current_metric.index)+1, y=current_metric.distance)
    
    temp_fig.update_axis()
    fig_metric = temp_fig.get_figure()
    
    return fig_metric

### Layout

In [6]:
bound_traj = pn.bind(plot_trajectory, widg_traj, widg_steps, widg_x_pos, widg_y_pos, widg_speed, widg_cauchy, widg_levy_alpha, widg_levy_beta)
bound_metric = pn.bind(plot_metric, widg_metric, widg_traj, widg_steps, widg_x_pos, widg_y_pos, widg_speed, widg_cauchy, widg_levy_alpha, widg_levy_beta)

param_column = pn.Column("### Trajectory type", widg_traj, pn.bind(update_layout, widg_traj), widg_metric, styles=dict(background='WhiteSmoke'))
traj_column = pn.Column("### Trajectory plot", pn.pane.Plotly(bound_traj, height=500, width=500), styles=dict(background='WhiteSmoke'))
metric_column = pn.Column("### Metric plot", pn.pane.Plotly(bound_metric, height=500, width=500), styles=dict(background='WhiteSmoke'))

layout = pn.Row(param_column, traj_column, metric_column)
layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'d736225a-bfac-4f7b-b6d1-ffcb93665d02': {'version…